<a href="https://colab.research.google.com/github/Neoneto/CodingDojo_Project_1/blob/main/Project_1_Part_2_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Project 1 - Part 1 (Core)
Submitted by Kenneth Alaba



## Imports the necessary dataset as pandas dataframe


In [94]:
# Import the pandas library
import pandas as pd

In [95]:
# Loads the data 
filename = '/content/sales_predictions.csv'
df_sales = pd.read_csv(filename)

# Shows the head of the dataframe
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Displays the information of the dataframe

In [96]:
# Shows the info of the dataframe
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


From here, we can see the data type that each column contains. We can also see that two of the columns contain missing values.

## Displays the duplicated rows

In [97]:
# Shows the duplicated rows of the dataframe
df_sales[df_sales.duplicated()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


This tells us that the dataframe contains no duplicated rows.

## Missing values

In [98]:
# Shows how many missing values are in each columns
df_sales.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

This tells us that only the columns 'Item_Weight' and 'Outlet_Size' has missing values which we will try to fill in the later parts.

## Dealing with missing values
Since all the missing values are only in the columns of the Item_Weight  and Outlet_Size , we can easily fill those missing values by looking at other rows with the same Item_Identifier for the weight and same Outlet_Identifier for the outlet size and just copying their values. We can do this since both the Item_Identifier and the Outlet_Identifier are unique.

### Filling missing values in Item_Weight

To fill the missing weights, we create a dictionary of the product identifier, which is unique, and its corresponding weight, using entries from other rows. We then map this dictionary to fill the missing values depending on the item identifier of the missing weight.

In [99]:
# Gets the set (unique values) of the product identifiers and its weight
dict_weight = df_sales[['Item_Identifier','Item_Weight']].dropna()
dict_weight = dict_weight.drop_duplicates()

# Creates a dictionary from the set
dict_weight = dict_weight.set_index('Item_Identifier')['Item_Weight'].to_dict()


In [100]:
# Fills the missing values in Item_Weight using the dictionary
df_sales.Item_Weight = df_sales.Item_Weight.fillna(df_sales.Item_Identifier.map(dict_weight))

# Checks whether there are still missing values
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


We can see that there are still missing values in the Item_Weight column.

In [101]:
# Obtain the rows with still missing weights
missing_weight = df_sales[df_sales[['Item_Identifier','Item_Weight']].isnull().any(axis=1)]
missing_weight

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [102]:
# Checks whether those Item_Identifier really doesn't appear in the dataframe
# with its corresponding weight.
for identifier in missing_weight['Item_Identifier']:
  print(identifier in df_sales[['Item_Identifier','Item_Weight']].dropna().drop_duplicates())

False
False
False
False


Since we can't use other entries to readily fill them, we will deal with them later.

### Filling missing values in Outlet_Size

Similar with the above method, we create a dictionary of Outlet_Identifier and its corresponding Outlet_Size and use that dictionary to fill the missing values.

In [103]:
# Create the dictionary from unique values in the Outlet_Identifier and Outlet_Size columns
dict_size = df_sales[['Outlet_Identifier','Outlet_Size']].dropna()
dict_size = dict_size.drop_duplicates()
dict_size = dict_size.set_index('Outlet_Identifier')['Outlet_Size'].to_dict()

In [104]:
# Fills the missing values depending on the Outlet_Identifier using the dictionary
df_sales.Outlet_Size = df_sales.Outlet_Size.fillna(df_sales.Outlet_Identifier.map(dict_size))

# Checks whether there are still missing values
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


We can see that there are still missing values, in fact, the dictionary wasn't able to fill any missing values in the Outlet_Size column because those Outlet_Identifier never appeared in the dataframe with its corresponding size as shown using the code below.

In [105]:
# Isolates the columns with missing Outlet_Size
missing_size = df_sales[df_sales[['Outlet_Identifier','Outlet_Size']].isnull().any(axis=1)]
missing_size = missing_size[['Outlet_Identifier','Outlet_Size']].drop_duplicates()

In [106]:
# Checks if the Outlet_Identifier appeared in the dataframe with its Outlet_Size
for identifier in missing_size['Outlet_Identifier']:
  print(identifier in df_sales[['Outlet_Identifier','Outlet_Size']].dropna().drop_duplicates())

False
False
False


### Filling further missing values

Since we don't know what we wan't to extract from the dataframe yet, I think that the best way to deal with those remaining missing values is to leave them as is for now until later where we know what we want to do and can determine the most appropriate method to fill them. However, for the sake of this excercise, since the Item_Weight is numerical, the best way to fill them is by using the mean of the column(while ignoring the missing values). As for the Outlet_Size, since it is categorical, the best way is to fill it with the most frequent value in the column.

In [107]:
# Obtain the most frequent value in the Outlet_Size column
frequent_size = df_sales.Outlet_Size.mode()[0]

In [108]:
# Fill the missing values in the Outlet_Size using that most frequent value
df_sales.Outlet_Size = df_sales.Outlet_Size.fillna(frequent_size)

In [109]:
# Calculate the mean weight of the items
mean_weight = df_sales['Item_Weight'].mean(skipna=True) # Ignores missing values

# Fills missing values using the mean weight
df_sales.Item_Weight = df_sales.Item_Weight.fillna(mean_weight)

In [110]:
# Checks that there are no more missing values
df_sales.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

## Fixing inconsistent categories

By manual inspection, we can see that the columns that can possibly suffer from inconsistency are the categorical ones which are listed in the list below.

In [111]:
# Creates a list of the columns with possible inconsistency
categorical = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

In [112]:
# Shows the unique values in each column of interest
for column in categorical:
  print(df_sales[column].drop_duplicates())

0     Low Fat
1     Regular
27    low fat
45         LF
81        reg
Name: Item_Fat_Content, dtype: object
0                      Dairy
1                Soft Drinks
2                       Meat
3      Fruits and Vegetables
4                  Household
5               Baking Goods
6                Snack Foods
8               Frozen Foods
15                 Breakfast
16        Health and Hygiene
18               Hard Drinks
29                    Canned
33                    Breads
72             Starchy Foods
139                   Others
231                  Seafood
Name: Item_Type, dtype: object
0     Medium
4       High
11     Small
Name: Outlet_Size, dtype: object
0    Tier 1
1    Tier 3
8    Tier 2
Name: Outlet_Location_Type, dtype: object
0    Supermarket Type1
1    Supermarket Type2
3        Grocery Store
7    Supermarket Type3
Name: Outlet_Type, dtype: object


By manual inspection, we can see that only the column Item_Fat_Content has inconsistencies where some Low Fat values are keyed in as either LF or low fat and Regular as reg. 

In [113]:
# Replaces inconsistencies in Item_Fat_Content column with their proper value
df_sales = df_sales.replace({'Item_Fat_Content': {'LF': 'Low Fat',
                                         'low fat': 'Low Fat',
                                         'reg': 'Regular'}})


In [114]:
# Checks that there are no more inconsistencies
for column in categorical:
  print(df_sales[column].drop_duplicates())

0    Low Fat
1    Regular
Name: Item_Fat_Content, dtype: object
0                      Dairy
1                Soft Drinks
2                       Meat
3      Fruits and Vegetables
4                  Household
5               Baking Goods
6                Snack Foods
8               Frozen Foods
15                 Breakfast
16        Health and Hygiene
18               Hard Drinks
29                    Canned
33                    Breads
72             Starchy Foods
139                   Others
231                  Seafood
Name: Item_Type, dtype: object
0     Medium
4       High
11     Small
Name: Outlet_Size, dtype: object
0    Tier 1
1    Tier 3
8    Tier 2
Name: Outlet_Location_Type, dtype: object
0    Supermarket Type1
1    Supermarket Type2
3        Grocery Store
7    Supermarket Type3
Name: Outlet_Type, dtype: object


## Summarize numerical columns

We isolate the numerical columns by observing what each column means and the data type of each. Just because the data type is integer or float etc. does not necessarily mean that the column is numerical.(i.e. some product ID can be int but are still nominal variables)

In [116]:
# Isolate the numerical columns
num_df = df_sales.select_dtypes(include=['float64', 'int64' ])

# Display the head()
num_df.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
0,9.30,0.016047,249.8092,1999,3735.1380
1,5.92,0.019278,48.2692,2009,443.4228
2,17.50,0.016760,141.6180,1999,2097.2700
3,19.20,0.000000,182.0950,1998,732.3800
4,8.93,0.000000,53.8614,1987,994.7052


In [117]:
# Calculates the mean of each column
newdf.mean()

Item_Weight                    12.875420
Item_Visibility                 0.066132
Item_MRP                      140.992782
Outlet_Establishment_Year    1997.831867
Item_Outlet_Sales            2181.288914
dtype: float64

In [118]:
# Find the minimum value for each column
newdf.min()

Item_Weight                     4.555
Item_Visibility                 0.000
Item_MRP                       31.290
Outlet_Establishment_Year    1985.000
Item_Outlet_Sales              33.290
dtype: float64

In [119]:
# Find the maximum value for each column
newdf.max()

Item_Weight                     21.350000
Item_Visibility                  0.328391
Item_MRP                       266.888400
Outlet_Establishment_Year     2009.000000
Item_Outlet_Sales            13086.964800
dtype: float64